## Fixed End Forces
This module computes the fixed end forces (moments and shears) due to transverse loads
acting on a 2-D planar structural member.

In [1]:
from __future__ import division, print_function

import numpy as np
import sys
from salib import *

In [2]:
class FEF(object):
    
    def __init__(self,c0=0.,v1=0.,m2=0.,c3=0.,v4=0.,m5=0.):
        if type(c0) is np.ndarray:
            self.fefs = c0.copy()
        else:
            self.fefs = np.array([c0,v1,m2,c3,v4,m5],dtype=np.float64)
        
    def __getitem__(self,ix):
        return self.fefs[ix]
    
    def __add__(self,other):
        assert type(self) is type(other)
        new = FEF(self.fefs+other.fefs)
        return new
    
    def __repr__(self):
        return '{}({},{},{},{},{},{})'.format(self.__class__.__name__,*(list(self.fefs)))

In [3]:
##test:
f = FEF(1,2,0,4,1,6)
f

FEF(1.0,2.0,0.0,4.0,1.0,6.0)

In [4]:
##test:
g = f+f+f
g

FEF(3.0,6.0,0.0,12.0,3.0,18.0)

In [5]:
##test:
f[1]

2.0

In [6]:
##test:
g[np.ix_([3,0,1])]

array([ 12.,   3.,   6.])

In [7]:
##test:
g[([3,0,1],)]

array([ 12.,   3.,   6.])

In [8]:
##test:
f0,f1,f2,f3,f4,f5 = g
f3

12.0

In [9]:
@extend(FEF)
class FEF:

    @property
    def c0(self):
        return self.fefs[0]
    
    @c0.setter
    def c0(self,v):
        self.fefs[0] = v
    
    @property
    def v1(self):
        return self.fefs[1]
    
    @v1.setter
    def v1(self,v):
        self.fefs[1] = v
    
    @property
    def m2(self):
        return self.fefs[2]
    
    @m2.setter
    def m2(self,v):
        self.fefs[2] = v
    
    @property
    def c3(self):
        return self.fefs[3]
    
    @c3.setter
    def c3(self,v):
        self.fefs[3] = v
    
    @property
    def v4(self):
        return self.fefs[4]
    
    @v4.setter
    def v4(self,v):
        self.fefs[4] = v
    
    @property
    def m5(self):
        return self.fefs[5]
    
    @m5.setter
    def m5(self,v):
        self.fefs[5] = v

In [10]:
#test:
f = FEF(10.,11,12,13,15,15)
f, f.c0, f.v1, f.m2, f.c3, f.v4, f.m5

(FEF(10.0,11.0,12.0,13.0,15.0,15.0), 10.0, 11.0, 12.0, 13.0, 15.0, 15.0)

In [11]:
#test:
f.c0 *= 2
f.v1 *= 3
f.m2 *= 4
f.c3 *= 5
f.v4 *= 6
f.m5 *= 7
f

FEF(20.0,33.0,48.0,65.0,90.0,105.0)

In [12]:
class MemberLoad(object):
    
    def fefs(self):
        """Return the complete set of 6 fixed end forces produced by the load."""
        raise NotImplementedError()
        
    def shear(self,x):
        """Return the shear force that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-dimensional array
        of values."""
        raise NotImplementedError()
        
    def moment(self,x):
        """Return the bending moment that is in equilibrium with that
        produced by the portion of the load to the left of the point at 
        distance 'x'.  'x' may be a scalar or a 1-dimensional array
        of values."""
        raise NotImplementedError()

In [13]:
@extend(MemberLoad)
class MemberLoad:
    
    @property
    def vpts(self):
        """Return a descriptor of the points at which the shear force must 
        be evaluated in order to draw a proper shear force diagram for this 
        load.  The descriptor is a 3-tuple of the form: (l,r,d) where 'l'
        is the leftmost point, 'r' is the rightmost point and 'd' is the
        degree of the curve between.  One of 'r', 'l' may be None."""
        raise NotImplementedError()
    
    @property
    def mpts(self):
        """Return a descriptor of the points at which the moment must be 
        evaluated in order to draw a proper bending moment diagram for this 
        load.  The descriptor is a 3-tuple of the form: (l,r,d) where 'l'
        is the leftmost point, 'r' is the rightmost point and 'd' is the
        degree of the curve between.  One of 'r', 'l' may be None."""
        raise NotImplementedError()

In [14]:
class PL(MemberLoad):
    
    def __init__(self,L,P,a):
        self.L = L
        self.P = P
        self.a = a
        
    def fefs(self):
        P = self.P
        L = self.L
        a = self.a
        b = L-a
        m2 = -P*a*b*b/(L*L)
        m5 = P*a*a*b/(L*L)
        v1 = (m2 + m5 - P*b)/L
        v4 = -(m2 + m5 + P*a)/L
        return FEF(0.,v1,m2,0.,v4,m5)
    
    def shear(self,x):
        return -self.P*(x>self.a)
    
    def moment(self,x):
        return self.P*(x-self.a)*(x>self.a)
        
    def __repr__(self):
        return '{}(L={},W1={},a={})'.format(self.__class__.__name__,self.L,self.P,self.a)

In [15]:
#test:
p = PL(1000.,300.,400.)
p, p.fefs()

(PL(L=1000.0,W1=300.0,a=400.0), FEF(0.0,-194.4,-43200.0,0.0,-105.6,28800.0))

In [16]:
@extend(MemberLoad)
class MemberLoad:
    
    EPSILON = 1.0E-6

@extend(PL)
class PL:
    
    @property
    def vpts(self):
        return (self.a-self.EPSILON,self.a+self.EPSILON,0)
    
    @property
    def mpts(self):
        return (self.a,None,1)

In [17]:
p = PL(1000.,300.,400.)
p.vpts

(399.999999, 400.000001, 0)

In [18]:
p.mpts

(400.0, None, 1)